In [127]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [128]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'a') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [129]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]

In [130]:
import chardet

def read_file_content(file_path):
    # Detect the encoding
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
    
    # Read the file with the detected encoding
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
        return content
    except FileNotFoundError:
        return f"Error: The file at path {file_path} was not found."
    except UnicodeDecodeError:
        return f"Error: The file at path {file_path} cannot be decoded with the {encoding} encoding."
    except IOError:
        return f"Error: An I/O error occurred while reading the file at path {file_path}."


In [131]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [132]:
TEST_CASE_GENERATOR="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [133]:
def sanitize_string(data_str):
    # Replace single quotes with double quotes
    data_str = data_str.replace("'", '"')
    
    # Escape double quotes inside the values
    in_string = False
    sanitized_str = ""
    for i in range(len(data_str)):
        if data_str[i] == '"':
            if in_string and (i == 0 or data_str[i-1] != '\\'):
                sanitized_str += '\\"'
            else:
                sanitized_str += '"'
            in_string = not in_string if data_str[i] == '"' and (i == 0 or data_str[i-1] != '\\') else in_string
        else:
            sanitized_str += data_str[i]
    return sanitized_str

In [134]:
use_case_directory_path = r"D:\Dissertation-GPT\dataset\SpecificationData\MatchaEnglishWebsite"
save_path = r"D:\Dissertation-GPT\evaluate\baseline 3 1.8\MatchaEnglishWebsite"

In [135]:
    # "Add new topic": [
    #     "1. Successful Topic Addition",
    #     "2. Adding Topic with Duplicate Name",
    #     "3. Cancel Topic Addition",
    #     "4. Adding Topic without Name",
    #     "5. Adding Topic without Description",
    #     "6. Adding Topic with Special Characters in Name",
    #     "7. Adding Topic with Long Name",
    #     "8. Adding Topic with Long Description"
    # ],
    # "AddLesson": [
    #     "1. Successful Lesson Addition",
    #     "2. Adding Lesson with Already Used Name",
    #     "3. Cancel Adding New Lesson",
    #     "4. Navigate to Add New Lesson Page via Navbar Option",
    #     "5. Adding Lesson Without Name",
    #     "6. Adding Lesson Without Description",
    #     "7. Adding Lesson with Special Characters in Name",
    #     "8. Adding Lesson with Long Name and Description",
    #     "9. Adding Lesson with HTML Tags in Name or Description",
    #     "10. Adding Lesson with Script Injection in Name or Description"
    # ],
    # "AddQuestion ": [
    #     "1. Successful question creation for selection test",
    #     "2. Access Lesson Management from Homepage",
    #     "3. Skip compulsory field \"sentence\" and handle error",
    #     "4. Skip compulsory field \"correct answer\" and handle error",
    #     "5. Skip compulsory field \"other option 1\" and handle error",
    #     "6. Skip compulsory field \"other option 2\" and handle error",
    #     "7. Skip compulsory field \"other option 3\" and handle error",
    #     "8. Cancel question creation and choose to stay",
    #     "9. Enter invalid sentence and handle error",
    #     "10. Leave page without saving question",
    #     "11. Cancel question creation and confirm cancellation"
    # ],
    # "EditTest": [
    #     "1. Successful Test Edit",
    #     "2. Cancel Edit and Exit Without Saving",
    #     "5. Save Edited Test With Mandatory Fields Missing",
    #     "7. Edit Test With Special Characters in Vocabulary",
    #     "8. Edit Test With Maximum Length of Vocabulary",
    #     "9. Edit Test With Minimum Length of Vocabulary"
    # ],
    # "Flashcard": [
    #     "1. Successful Navigation and Learning Flow",
    #     "2. Navigation with Back Button and Stay Decision",
    #     "3. Continue Studying After Completion",
    #     "4. Direct Access to Incomplete Topic from Home Page",
    #     "5. Attempt to Navigate to Previous Flashcard on First Card",
    #     "6. Exit During Flashcard Study and Mark Topic as Incomplete"
    # ],
    # "Handbook": [
    #     "1. View Vocabulary List",
    #     "2. Scroll Vocabulary List",
    #     "3. Search Vocabulary Successfully",
    #     "4. Search Meaningless Vocabulary",
    #     "5. Exclude Vocabulary from Review Test",
    #     "6. Include Vocabulary in Review Test"
    # ],

    # "Learning days history": [
    #     "1. **View Continuous Learning Days Scenario** - This scenario covers the basic flow where a learner checks their continuous learning days without any interruptions in their learning streak.",
    #     "2. **View Learning Days After Break Scenario** - This scenario tests the functionality when a learner has recently broken their learning streak and checks their learning days, which should reset to one."
    # ],
usecase_scenario = {

    "ListenToPronunciation": [
        "1. Successful Pronunciation Test Completion",
        "2. Incorrect Answer Selection",
        "6. Attempting Test Without Being Logged In",
        "8. Refreshing Page During Test",
        "9. Closing Application During Test",
        "11. Skipping Pronunciation Test Question"
    ],
    "Login": [
        "1. Successful Login with Username",
        "2. Successful Login with Email",
        "3. Login with Wrong Password",
        "4. Login with Unregistered Username",
        "5. Login with Blank Username Field",
        "6. Login with Blank Password Field"
    ],
    "Registry": [
        "1. Successful Registration with Valid Username and Password",
        "2. Successful Registration with Email",
        "4. Registration with Invalid Password",
        "5. Registration with Existed Username",
        "7. Registration with Password Same as Username",
        "8. Registration with Username Below Minimum Length",
        "9. Registration with Username Above Maximum Length",
        "10. Registration with Password Below Minimum Length",
        "11. Registration with Password Above Maximum Length",
        "12. Registration with Email in Incorrect Format",
        "13. Registration with Non-Existent Email Address"
    ],
    "Review test": [
        "1. Successful Review Completion with Correct Answers",
        "2. Review Completion with Incorrect Answers",
        "3. Review Process with Mixed Correct and Incorrect Answers",
        "4. Exiting Review and Returning to Continue",
        "5. Exiting Review and Confirming Exit",
        "6. Review with Questions Method",
        "7. Review with Sound Method",
        "8. Review with Image Method",
        "9. Review with Meanings Method",
        "10. Last Word Review and Return to Home Page",
        "11. Review with No Words in Review List"
    ],
    "WordSortingTest": [
        "1. Successful Word Sorting with Vocabulary Definition",
        "2. Successful Word Sorting with Image Clue",
        "3. Incorrect Word Sorting with Vocabulary Definition",
        "4. Incorrect Word Sorting with Image Clue",
        "8. Word Sorting with Fewer Letters than Required"
    ]
}

In [136]:
usecase_scenario = {    
    "Learning days history": [
        "1. **View Continuous Learning Days Scenario** ",
        "2. **View Learning Days After Break Scenario** "
    ],
}

In [137]:
# usecase_scenario = """{'Add new topic': ['1. Successful Topic Addition', '2. Adding Topic with Duplicate Name', '3. Cancel Topic Addition', '4. Adding Topic without Name', '5. Adding Topic without Description', '6. Adding Topic with Special Characters in Name', '7. Adding Topic with Long Name', '8. Adding Topic with Long Description'], 'AddLesson': ['1. Successful Lesson Addition', '2. Adding Lesson with Already Used Name', '3. Cancel Adding New Lesson', '4. Navigate to Add New Lesson Page via Navbar Option', '5. Adding Lesson Without Name', '6. Adding Lesson Without Description', '7. Adding Lesson with Special Characters in Name', '8. Adding Lesson with Long Name and Description', '9. Adding Lesson with HTML Tags in Name or Description', '10. Adding Lesson with Script Injection in Name or Description'], 'AddQuestion - Copy': ['1. Successful Question Creation', '2. Access Lesson Management from Homepage', '3. Skip Compulsory Field "Sentence"', '4. Skip Compulsory Field "Correct Answer"', '5. Skip Compulsory Field "Other Option 1"', '6. Skip Compulsory Field "Other Option 2"', '7. Skip Compulsory Field "Other Option 3"', '8. Cancel Question Creation and Choose to Stay', '9. Enter Invalid Field Input', '10. Leave Page Without Saving Question', '11. Cancel Question Creation and Confirm Cancelation'], 'AddQuestion ': ['1. Successful question creation for selection test', '2. Access Lesson Management from Homepage', '3. Skip compulsory field "sentence" and handle error', '4. Skip compulsory field "correct answer" and handle error', '5. Skip compulsory field "other option 1" and handle error', '6. Skip compulsory field "other option 2" and handle error', '7. Skip compulsory field "other option 3" and handle error', '8. Cancel question creation and choose to stay', '9. Enter invalid sentence and handle error', '10. Leave page without saving question', '11. Cancel question creation and confirm cancellation'], 'EditTest': ['1. Successful Test Edit', '2. Cancel Edit and Exit Without Saving', '5. Save Edited Test With Mandatory Fields Missing', '7. Edit Test With Special Characters in Vocabulary', '8. Edit Test With Maximum Length of Vocabulary', '9. Edit Test With Minimum Length of Vocabulary'], 'Flashcard': ['1. Successful Navigation and Learning Flow', '2. Navigation with Back Button and Stay Decision', '3. Continue Studying After Completion', '4. Direct Access to Incomplete Topic from Home Page', '5. Attempt to Navigate to Previous Flashcard on First Card', '6. Exit During Flashcard Study and Mark Topic as Incomplete'], 'Handbook': ['1. View Vocabulary List', '2. Scroll Vocabulary List', '3. Search Vocabulary Successfully', '4. Search Meaningless Vocabulary', '5. Exclude Vocabulary from Review Test', '6. Include Vocabulary in Review Test'], 'Learning days history': ['1. **View Continuous Learning Days Scenario** - This scenario covers the basic flow where a learner checks their continuous learning days without any interruptions in their learning streak.', '2. **View Learning Days After Break Scenario** - This scenario tests the functionality when a learner has recently broken their learning streak and checks their learning days, which should reset to one.'], 'ListenToPronunciation': ['1. Successful Pronunciation Test Completion', '2. Incorrect Answer Selection', '6. Attempting Test Without Being Logged In', '8. Refreshing Page During Test', '9. Closing Application During Test', '11. Skipping Pronunciation Test Question'], 'Login': ['1. Successful Login with Username', '2. Successful Login with Email', '3. Login with Wrong Password', '4. Login with Unregistered Username', '5. Login with Blank Username Field', '6. Login with Blank Password Field'], 'Registry': ['1. Successful Registration with Valid Username and Password', '2. Successful Registration with Email', '4. Registration with Invalid Password', '5. Registration with Existed Username', '7. Registration with Password Same as Username', '8. Registration with Username Below Minimum Length', '9. Registration with Username Above Maximum Length', '10. Registration with Password Below Minimum Length', '11. Registration with Password Above Maximum Length', '12. Registration with Email in Incorrect Format', '13. Registration with Non-Existent Email Address'], 'Review test': ['1. Successful Review Completion with Correct Answers', '2. Review Completion with Incorrect Answers', '3. Review Process with Mixed Correct and Incorrect Answers', '4. Exiting Review and Returning to Continue', '5. Exiting Review and Confirming Exit', '6. Review with Questions Method', '7. Review with Sound Method', '8. Review with Image Method', '9. Review with Meanings Method', '10. Last Word Review and Return to Home Page', '11. Review with No Words in Review List'], 'WordSortingTest': ['1. Successful Word Sorting with Vocabulary Definition', '2. Successful Word Sorting with Image Clue', '3. Incorrect Word Sorting with Vocabulary Definition', '4. Incorrect Word Sorting with Image Clue', '8. Word Sorting with Fewer Letters than Required']}"""
# data_str = usecase_scenario.replace("'", '"')
# usecase_scenario_json = json.loads(sanitize_string(data_str))

In [138]:
for key, value in usecase_scenario.items():
  usecase = key
  usecase_name =usecase
  usecase_path = os.path.join (use_case_directory_path,f"{usecase_name}.txt" )
  for testscenario in value:
    content = "Test scenarios:"+ testscenario+  "\nUse case:" + read_file_content(usecase_path)
    print(content)
    promptExtractCondition = [
      { "role": "system", "content": TEST_CASE_GENERATOR},
      { "role": "user", "content": content}
    ]
    gpt_response = askJSON(promptExtractCondition, client, model)
    print(gpt_response)
    print()
    json_data = json.loads(gpt_response)
    testscenario = testscenario.replace('"','').replace(':','').replace('?','').replace('*','').replace('/','').replace('.','')
    testcase_path = os.path.join (save_path,f"{usecase_name}-{testscenario}.txt" )
    write_json_to_txt_raw(json_data,testcase_path )
  print("f{usecase_name} gen test case complete")

Test scenarios:1. **View Continuous Learning Days Scenario** 
Use case:use case id: U11
use case name: learning days history
objective: This use case allows to see the days that learner has spent learning.
actor: learner

basic flow:
Step 1: Learner selects "History" on Home page.
Step 2: The system redirects learner to History page.
Step 3: Learner selects "Day History" on History page.
Step 4: The system redirects learner to a page show how many days learner has learned continuously in number. If learner break their chain, the number reset to one.

Preconditions:
Learner must be logged into the application.

Postconditions:
System shows the days learner has spent learning. 
{
  "Test Case 1": {
    "testCaseName": "Verify Continuous Learning Days Display",
    "objective": "Verify that the system correctly displays the number of continuous learning days for a learner without breaking the learning chain.",
    "testSteps": [
      "Step 1: Ensure the learner is logged into the applica